In [1]:
import os
pyspark_submit_args = "--jars spark-streaming-kafka-0-8-assembly_2.11-2.1.0.jar"
pyspark_submit_args += " --driver-memory 4g"
pyspark_submit_args += " --executor-memory 4g"
pyspark_submit_args += " pyspark-shell"
os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args

In [2]:
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils, TopicAndPartition

In [3]:
spark = SparkSession.builder.master("local[*]").appName("createStream").getOrCreate()
sc = spark.sparkContext
stc = StreamingContext(sc, 30)

In [4]:
brokers, topic = "localhost:9092", "test"
fromOffsets = {TopicAndPartition(topic, 0): int(0)}
kvs = KafkaUtils.createDirectStream(
    stc, [topic], {"metadata.broker.list": brokers}, 
    fromOffsets=fromOffsets
)

In [ ]:
wordCounts = (
    kvs
        .map(lambda x: x[1])
        .flatMap(lambda line: line.split(" "))
        .map(lambda word: (word, 1))
        .reduceByKey(lambda x, y: x + y)
)
wordCounts.pprint()

In [ ]:
stc.start()
stc.awaitTermination()

-------------------------------------------
Time: 2018-12-14 10:30:30
-------------------------------------------
('spark', 10)
('a', 4)
('b', 3)
('c', 3)
('stream', 17)
('strema', 1)
('kafka', 19)

-------------------------------------------
Time: 2018-12-14 10:31:00
-------------------------------------------
('go', 4)
('up', 1)
('down', 3)

-------------------------------------------
Time: 2018-12-14 10:31:30
-------------------------------------------
('left', 2)
('go', 3)
('right', 2)

-------------------------------------------
Time: 2018-12-14 10:32:00
-------------------------------------------

